# Module 1: Protein Structure Representation

**📍 Notebook 3 of 8**

## 💻 GPU Requirements
**✅ No GPU needed!** All code runs on CPU.

---

## 🎯 Learning Objectives

By the end of this notebook, you will:

1. Understand protein structure hierarchy (primary → quaternary)
2. Know the key atoms in protein backbones (N, CA, C, O)
3. Work with PDB file format
4. Extract and manipulate backbone coordinates
5. Understand internal coordinates (bonds, angles, dihedrals)
6. Learn about rigid body frames and transformations
7. Prepare protein data for RFDiffusion input

## 📚 What You'll Learn

RFDiffusion doesn't work with full atomic detail - it operates on **backbone representations**. Understanding how to encode proteins efficiently is crucial!

---

## 🧬 Protein Structure Hierarchy

Proteins have four levels of structure:

### 1. Primary Structure
The **sequence** of amino acids (20 types).
```
MAKVLGD... (one-letter codes)
```

### 2. Secondary Structure
Local patterns: **α-helices**, **β-sheets**, **loops**

### 3. Tertiary Structure
Full 3D fold of a single chain

### 4. Quaternary Structure
Multiple chains assembled together

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from Bio.PDB import PDBParser, PDBIO, Select
from Bio.PDB.vectors import Vector, rotmat
import os

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
np.set_printoptions(precision=3, suppress=True)

print("✅ Libraries loaded!")

## ⚛️ Backbone Atoms: The Core Representation

Each amino acid (residue) has a **backbone** and **sidechain**.

### Backbone Atoms (same for all amino acids):
- **N** (nitrogen) - amide nitrogen
- **CA** (alpha carbon) - central carbon
- **C** (carbonyl carbon) - carbonyl carbon  
- **O** (oxygen) - carbonyl oxygen

### Why Backbone-Only?

**RFDiffusion uses only CA atoms** (sometimes N, CA, C):

**Advantages:**
- ✅ Simpler representation (1 point per residue vs ~10-20 atoms)
- ✅ Captures overall fold
- ✅ Faster computation
- ✅ Easier to generate valid structures

**Later:** Sidechains can be added with ProteinMPNN or other tools

### Backbone Representation:
```
Residue 1:     Residue 2:     Residue 3:
N-CA-C-O  →  N-CA-C-O  →  N-CA-C-O
```

For RFDiffusion:
```
CA₁ → CA₂ → CA₃ → ... → CAₙ
```

In [ ]:
# Load a protein structure
from Bio.PDB import PDBList

# Download a small protein if not already present
pdb_dir = '../data/examples'
os.makedirs(pdb_dir, exist_ok=True)

pdb_id = '1VII'  # Villin headpiece (36 residues)
pdb_file = f'{pdb_dir}/pdb{pdb_id.lower()}.ent'

if not os.path.exists(pdb_file):
    print(f"Downloading {pdb_id}...")
    pdbl = PDBList()
    pdbl.retrieve_pdb_file(pdb_id, file_format='pdb', pdir=pdb_dir)
    print(f"✅ Downloaded!")
else:
    print(f"✅ {pdb_id} already downloaded")

# Parse the structure
parser = PDBParser(QUIET=True)
structure = parser.get_structure(pdb_id, pdb_file)
model = structure[0]
chain = list(model.get_chains())[0]
residues = [r for r in chain.get_residues() if r.id[0] == ' ']  # Standard residues only

print(f"\n📊 Structure Information:")
print(f"   PDB ID: {pdb_id}")
print(f"   Number of residues: {len(residues)}")
print(f"   Chain: {chain.id}")

# Extract backbone atoms
def get_backbone_atoms(residues):
    """Extract N, CA, C coordinates from residues."""
    n_coords = []
    ca_coords = []
    c_coords = []
    
    for res in residues:
        if res.has_id('N'):
            n_coords.append(res['N'].get_coord())
        if res.has_id('CA'):
            ca_coords.append(res['CA'].get_coord())
        if res.has_id('C'):
            c_coords.append(res['C'].get_coord())
    
    return np.array(n_coords), np.array(ca_coords), np.array(c_coords)

n_coords, ca_coords, c_coords = get_backbone_atoms(residues)

print(f"\n✅ Extracted backbone atoms:")
print(f"   N atoms:  {n_coords.shape}")
print(f"   CA atoms: {ca_coords.shape}")
print(f"   C atoms:  {c_coords.shape}")
print(f"\nFirst CA atom position: {ca_coords[0]}")
print(f"Last CA atom position:  {ca_coords[-1]}")

In [ ]:
# Visualize backbone atoms in 3D
fig = plt.figure(figsize=(15, 5))

# Plot 1: All backbone atoms
ax1 = fig.add_subplot(131, projection='3d')
ax1.plot(n_coords[:, 0], n_coords[:, 1], n_coords[:, 2], 'o-', label='N', markersize=4, alpha=0.6)
ax1.plot(ca_coords[:, 0], ca_coords[:, 1], ca_coords[:, 2], 'o-', label='CA', markersize=5, linewidth=2)
ax1.plot(c_coords[:, 0], c_coords[:, 1], c_coords[:, 2], 'o-', label='C', markersize=4, alpha=0.6)
ax1.set_xlabel('X (Å)')
ax1.set_ylabel('Y (Å)')
ax1.set_zlabel('Z (Å)')
ax1.set_title('All Backbone Atoms', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: CA trace only (what RFDiffusion uses)
ax2 = fig.add_subplot(132, projection='3d')
ax2.plot(ca_coords[:, 0], ca_coords[:, 1], ca_coords[:, 2], 'o-', 
         markersize=6, linewidth=2.5, color='#E63946')
ax2.set_xlabel('X (Å)')
ax2.set_ylabel('Y (Å)')
ax2.set_zlabel('Z (Å)')
ax2.set_title('CA Trace (RFDiffusion Input)', fontweight='bold')
ax2.grid(True, alpha=0.3)

# Plot 3: Distance between consecutive CAs
ax3 = fig.add_subplot(133)
ca_distances = np.linalg.norm(np.diff(ca_coords, axis=0), axis=1)
ax3.plot(ca_distances, 'o-', linewidth=2, markersize=6, color='#457B9D')
ax3.axhline(y=3.8, color='red', linestyle='--', linewidth=2, label='Ideal ~3.8Å')
ax3.fill_between(range(len(ca_distances)), 3.7, 3.9, alpha=0.2, color='red')
ax3.set_xlabel('Residue Index', fontsize=11)
ax3.set_ylabel('CA-CA Distance (Å)', fontsize=11)
ax3.set_title('CA-CA Distances', fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📏 CA-CA Distance Statistics:")
print(f"   Mean: {ca_distances.mean():.3f} Å")
print(f"   Std:  {ca_distances.std():.3f} Å")
print(f"   Min:  {ca_distances.min():.3f} Å")
print(f"   Max:  {ca_distances.max():.3f} Å")
print(f"\n💡 Key Point: CA-CA distance is ~3.8Å - a fundamental constraint!")

## 📐 Internal Coordinates: Bonds, Angles, Dihedrals

Instead of Cartesian coordinates (x, y, z), we can use **internal coordinates**:

### Three Types:

1. **Bond Lengths**
   - Distance between consecutive atoms
   - CA-CA: ~3.8Å (relatively constant)

2. **Bond Angles**  
   - Angle formed by 3 consecutive atoms
   - CA-CA-CA angle: ~110-120°

3. **Dihedral Angles (Torsion Angles)**
   - Angle of rotation around a bond
   - **φ (phi)** and **ψ (psi)** angles define backbone conformation
   - These are the most important for protein structure!

### Why Internal Coordinates?

- ✅ More compact (angles vs full 3D positions)
- ✅ Capture local geometry
- ✅ Can convert between Cartesian ↔ Internal
- ⚠️ But RFDiffusion works in Cartesian space (with SE(3) transforms)

In [ ]:
# Calculate dihedral angles
def calculate_dihedral(p1, p2, p3, p4):
    """
    Calculate dihedral angle between 4 points.
    
    Returns angle in degrees.
    """
    b1 = p2 - p1
    b2 = p3 - p2
    b3 = p4 - p3
    
    # Normalize b2
    b2_norm = b2 / np.linalg.norm(b2)
    
    # Calculate vectors perpendicular to b2
    v1 = b1 - np.dot(b1, b2_norm) * b2_norm
    v2 = b3 - np.dot(b3, b2_norm) * b2_norm
    
    # Calculate angle
    x = np.dot(v1, v2)
    y = np.dot(np.cross(b2_norm, v1), v2)
    
    return np.degrees(np.arctan2(y, x))

# Calculate phi and psi angles
def calculate_backbone_dihedrals(n_coords, ca_coords, c_coords):
    """Calculate phi and psi dihedral angles."""
    n_res = len(ca_coords)
    phi_angles = []
    psi_angles = []
    
    for i in range(1, n_res - 1):
        # Phi: C(i-1) - N(i) - CA(i) - C(i)
        if i > 0:
            phi = calculate_dihedral(c_coords[i-1], n_coords[i], ca_coords[i], c_coords[i])
            phi_angles.append(phi)
        
        # Psi: N(i) - CA(i) - C(i) - N(i+1)
        if i < n_res - 1:
            psi = calculate_dihedral(n_coords[i], ca_coords[i], c_coords[i], n_coords[i+1])
            psi_angles.append(psi)
    
    return np.array(phi_angles), np.array(psi_angles)

phi, psi = calculate_backbone_dihedrals(n_coords, ca_coords, c_coords)

# Ramachandran plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Scatter plot
ax1.scatter(phi, psi, s=100, alpha=0.7, c=range(len(phi)), cmap='viridis', edgecolors='black', linewidth=1)
ax1.set_xlabel('φ (Phi) angle (degrees)', fontsize=12)
ax1.set_ylabel('ψ (Psi) angle (degrees)', fontsize=12)
ax1.set_title('Ramachandran Plot', fontsize=13, fontweight='bold')
ax1.set_xlim(-180, 180)
ax1.set_ylim(-180, 180)
ax1.axhline(0, color='gray', linestyle='--', alpha=0.3)
ax1.axvline(0, color='gray', linestyle='--', alpha=0.3)
ax1.grid(True, alpha=0.3)

# Add typical regions
from matplotlib.patches import Rectangle
# Alpha helix region (approximate)
alpha_helix = Rectangle((-80, -60), 40, 60, alpha=0.2, color='red', label='α-helix')
ax1.add_patch(alpha_helix)
# Beta sheet region (approximate)
beta_sheet = Rectangle((-180, 100), 100, 80, alpha=0.2, color='blue', label='β-sheet')
ax1.add_patch(beta_sheet)
ax1.legend()

# Histogram
ax2.hist2d(phi, psi, bins=20, cmap='Blues')
ax2.set_xlabel('φ (Phi) angle (degrees)', fontsize=12)
ax2.set_ylabel('ψ (Psi) angle (degrees)', fontsize=12)
ax2.set_title('Ramachandran Density', fontsize=13, fontweight='bold')
ax2.set_xlim(-180, 180)
ax2.set_ylim(-180, 180)
plt.colorbar(ax2.collections[0], ax=ax2, label='Count')

plt.tight_layout()
plt.show()

print("📌 Ramachandran Plot:")
print("   - Shows allowed backbone conformations")
print("   - Clustering indicates secondary structure")
print("   - RFDiffusion must generate valid angles!")

## 🔄 Rigid Body Frames

**Key concept for RFDiffusion!**

Instead of representing each atom independently, we can use **local coordinate frames** attached to each residue.

### What's a Frame?

A frame is a coordinate system defined by:
- **Origin**: Position (e.g., CA atom)
- **Orientation**: Three orthogonal axes (x, y, z)

### Why Frames?

1. **SE(3) Equivariance**: Rotating/translating the whole protein = rotating/translating frames
2. **Compact**: Store rotation matrix (3×3) + translation (3×1) instead of all atom positions
3. **Natural**: Captures local geometry

### Frame Definition:

For each residue, we define a frame using N, CA, C atoms:
- **Origin**: CA position
- **x-axis**: Points from CA → C
- **y-axis**: Perpendicular to x, in plane of N-CA-C
- **z-axis**: Cross product of x and y

Then: Any atom position can be expressed relative to its local frame!

In [ ]:
# Build local frames for each residue
def build_local_frame(n, ca, c):
    """
    Build orthonormal frame from N, CA, C atoms.
    
    Returns:
        R: 3x3 rotation matrix (columns are x, y, z axes)
        t: 3x1 translation (CA position)
    """
    # Origin at CA
    t = ca
    
    # x-axis: CA -> C direction
    x = c - ca
    x = x / np.linalg.norm(x)
    
    # y-axis: perpendicular to x, in plane of N-CA-C
    v = n - ca
    y = v - np.dot(v, x) * x  # Remove component along x
    y = y / np.linalg.norm(y)
    
    # z-axis: perpendicular to both
    z = np.cross(x, y)
    
    # Rotation matrix (columns are axes)
    R = np.column_stack([x, y, z])
    
    return R, t

# Build frames for all residues
frames = []
for i in range(len(ca_coords)):
    R, t = build_local_frame(n_coords[i], ca_coords[i], c_coords[i])
    frames.append((R, t))

# Visualize a few frames
fig = plt.figure(figsize=(12, 5))

# Plot 1: First 5 residues with frames
ax1 = fig.add_subplot(121, projection='3d')
for i in range(min(5, len(frames))):
    R, t = frames[i]
    
    # Plot backbone
    ax1.plot(ca_coords[i:i+2, 0], ca_coords[i:i+2, 1], ca_coords[i:i+2, 2], 
             'o-', color='gray', markersize=8, linewidth=2, alpha=0.5)
    
    # Plot frame axes
    scale = 2.0
    origin = t
    ax1.quiver(*origin, *R[:, 0], color='red', length=scale, arrow_length_ratio=0.3, linewidth=2)  # x
    ax1.quiver(*origin, *R[:, 1], color='green', length=scale, arrow_length_ratio=0.3, linewidth=2)  # y
    ax1.quiver(*origin, *R[:, 2], color='blue', length=scale, arrow_length_ratio=0.3, linewidth=2)  # z

ax1.set_xlabel('X (Å)')
ax1.set_ylabel('Y (Å)')
ax1.set_zlabel('Z (Å)')
ax1.set_title('Local Frames (First 5 Residues)', fontweight='bold')

# Plot 2: Frame alignment check
ax2 = fig.add_subplot(122)
# Check orthonormality of first frame
R_test, _ = frames[0]
gram = R_test.T @ R_test  # Should be identity
ax2.imshow(gram, cmap='RdBu', vmin=-1, vmax=1)
ax2.set_title('Frame Orthonormality Check\n(Should be Identity)', fontweight='bold')
ax2.set_xlabel('Axis')
ax2.set_ylabel('Axis')
for i in range(3):
    for j in range(3):
        ax2.text(j, i, f'{gram[i,j]:.2f}', ha='center', va='center', fontsize=14)
plt.colorbar(ax2.images[0], ax=ax2)

plt.tight_layout()
plt.show()

print(f"✅ Built {len(frames)} local frames")
print(f"\nFrame 0 rotation matrix:")
print(frames[0][0])
print(f"\nFrame 0 translation (CA position):")
print(frames[0][1])

## 🎯 RFDiffusion Input Format

Now we know how proteins are represented! Here's what RFDiffusion actually uses:

### Input Representation:

For each residue i:
- **Frame**: Rotation matrix R_i (3×3) + Translation t_i (3×1)
- **Amino acid type**: One-hot encoded (20 types) - optional for unconditional generation

### Tensor Shapes:

```python
# For a protein with N residues:
frames_R = torch.tensor  # Shape: (N, 3, 3) - rotations
frames_t = torch.tensor  # Shape: (N, 3) - translations
aa_types = torch.tensor  # Shape: (N, 20) - amino acid types (optional)
```

### During Diffusion:

1. **Forward**: Add noise to frames (rotations + translations)
2. **Reverse**: Neural network predicts clean frames from noisy frames
3. **Output**: Valid protein backbone structure!

### Key Advantages:

✅ SE(3) equivariant - respects 3D symmetries  
✅ Compact representation  
✅ Captures local and global geometry  
✅ Easy to impose constraints (motifs, symmetry)

In [ ]:
# Convert to RFDiffusion format (PyTorch-like, but using NumPy here)
def protein_to_rfdiffusion_format(frames):
    """Convert list of frames to RFDiffusion tensor format."""
    n_residues = len(frames)
    
    # Initialize arrays
    rotations = np.zeros((n_residues, 3, 3))
    translations = np.zeros((n_residues, 3))
    
    for i, (R, t) in enumerate(frames):
        rotations[i] = R
        translations[i] = t
    
    return rotations, translations

rotations, translations = protein_to_rfdiffusion_format(frames)

print(f"✅ Converted to RFDiffusion format:")
print(f"   Rotations shape: {rotations.shape}")
print(f"   Translations shape: {translations.shape}")
print(f"\n📊 Memory comparison:")
print(f"   Full atom coords (N,CA,C): {n_coords.size + ca_coords.size + c_coords.size} floats")
print(f"   Frame representation: {rotations.size + translations.size} floats")
print(f"   Compression ratio: {(n_coords.size + ca_coords.size + c_coords.size) / (rotations.size + translations.size):.2f}x")

# Visualize frame representation
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Rotation matrices (first 10 residues)
ax1 = axes[0]
rot_flat = rotations[:10].reshape(10, 9)
im1 = ax1.imshow(rot_flat.T, aspect='auto', cmap='RdBu', vmin=-1, vmax=1)
ax1.set_xlabel('Residue')
ax1.set_ylabel('Rotation Matrix Elements')
ax1.set_title('Rotation Matrices (First 10)', fontweight='bold')
plt.colorbar(im1, ax=ax1)

# Translations
ax2 = axes[1]
ax2.plot(translations[:, 0], label='X', linewidth=2)
ax2.plot(translations[:, 1], label='Y', linewidth=2)
ax2.plot(translations[:, 2], label='Z', linewidth=2)
ax2.set_xlabel('Residue Index')
ax2.set_ylabel('Position (Å)')
ax2.set_title('CA Positions (Translations)', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Frame distances
ax3 = axes[2]
frame_dists = np.linalg.norm(np.diff(translations, axis=0), axis=1)
ax3.plot(frame_dists, 'o-', linewidth=2, markersize=6, color='#A23B72')
ax3.axhline(y=3.8, color='red', linestyle='--', linewidth=2, label='Ideal ~3.8Å')
ax3.fill_between(range(len(frame_dists)), 3.7, 3.9, alpha=0.2, color='red')
ax3.set_xlabel('Residue Index')
ax3.set_ylabel('Distance (Å)')
ax3.set_title('Frame-to-Frame Distances', fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 This is the actual input format RFDiffusion uses!")

## 🎓 Key Takeaways

1. **Proteins are represented by backbone atoms** (N, CA, C)
2. **RFDiffusion uses CA-trace** primarily (sometimes N, CA, C)
3. **CA-CA distance** is ~3.8Å - a key constraint
4. **Internal coordinates** (φ, ψ angles) define backbone conformation
5. **Rigid body frames** provide compact, SE(3)-equivariant representation
6. **Input format**: Rotation matrices + translations for each residue

## ✅ Self-Check Questions

1. What atoms make up the protein backbone?
2. Why does RFDiffusion use CA atoms instead of all atoms?
3. What are φ and ψ angles?
4. What is a Ramachandran plot?
5. What is a rigid body frame?
6. What are the dimensions of RFDiffusion's input tensors?

## 💡 Practice Exercise

Try:
1. Load a different protein from PDB
2. Calculate its backbone angles
3. Build local frames
4. Convert to RFDiffusion format

## 📖 Further Reading

- [Protein Data Bank (PDB)](https://www.rcsb.org/) - Protein structure database
- [Ramachandran Plot](https://en.wikipedia.org/wiki/Ramachandran_plot) - Backbone conformations
- [Rigid Body Transformations](https://en.wikipedia.org/wiki/Rigid_body) - SE(3) group

## ⏭️ Next Notebook

**04_se3_equivariance.ipynb** - Learn about geometric deep learning and SE(3) equivariance

💡 **Still no GPU needed!** Next notebook is about theory and geometry.